In [4]:
import random
import math
import matplotlib.pyplot as plt
from collections import deque, defaultdict


class CityMap:
    def __init__(self, num_nodes=30):
        self.nodes = list(range(num_nodes))
        self.map = defaultdict(list)
        self.pos = {}

        self._build_connected_graph(num_nodes)
        self._assign_positions()

    def _build_connected_graph(self, n):
        for i in range(n - 1):
            self.map[i].append(i + 1)
            self.map[i + 1].append(i)

        extra = int(n * 1.2)
        for _ in range(extra):
            a = random.randint(0, n - 1)
            b = random.randint(0, n - 1)
            if a != b and b not in self.map[a]:
                self.map[a].append(b)
                self.map[b].append(a)

    def _assign_positions(self):
        angle_step = 2 * math.pi / len(self.nodes)
        for i, node in enumerate(self.nodes):
            angle = i * angle_step
            self.pos[node] = (math.cos(angle), math.sin(angle))

    def bidirectional_bfs(self, start, goal):
        if start == goal:
            return [start], 0

        q1 = deque([start])
        q2 = deque([goal])

        from_start = {start: None}
        from_goal = {goal: None}

        explored = 0

        while q1 and q2:
            a = q1.popleft()
            explored += 1

            if a in from_goal:
                return self._join_paths(from_start, from_goal, a), explored

            for n in self.map[a]:
                if n not in from_start:
                    from_start[n] = a
                    q1.append(n)

            b = q2.popleft()
            explored += 1

            if b in from_start:
                return self._join_paths(from_start, from_goal, b), explored

            for n in self.map[b]:
                if n not in from_goal:
                    from_goal[n] = b
                    q2.append(n)

        return None, explored

    def _join_paths(self, left, right, mid):
        first = []
        cur = mid
        while cur is not None:
            first.append(cur)
            cur = left[cur]
        first.reverse()

        second = []
        cur = right[mid]
        while cur is not None:
            second.append(cur)
            cur = right[cur]

        return first + second

    def visualize(self, path, start, goal):
        plt.figure(figsize=(9, 7))

        for u in self.map:
            for v in self.map[u]:
                x1, y1 = self.pos[u]
                x2, y2 = self.pos[v]
                plt.plot([x1, x2], [y1, y2], color="lightgray", zorder=1)

        for node, (x, y) in self.pos.items():
            plt.scatter(x, y, color="lightblue", s=400, zorder=2)
            plt.text(x, y, str(node), ha="center", va="center", fontweight="bold")

        if path:
            for i in range(len(path) - 1):
                x1, y1 = self.pos[path[i]]
                x2, y2 = self.pos[path[i + 1]]
                plt.plot([x1, x2], [y1, y2], color="red", linewidth=2.5, zorder=3)

            for node in path:
                x, y = self.pos[node]
                plt.scatter(x, y, color="orange", s=450, zorder=4)

            sx, sy = self.pos[start]
            gx, gy = self.pos[goal]
            plt.scatter(sx, sy, color="green", s=500, zorder=5)
            plt.scatter(gx, gy, color="red", s=500, zorder=5)

        plt.title(f"Route from {start} to {goal}")
        plt.axis("off")
        plt.show()


if __name__ == "__main__":
    city = CityMap(num_nodes=25)

    start = 0
    end = 15

    path, count = city.bidirectional_bfs(start, end)

    print("Path found:", path)

    if path:
        city.visualize(path, start, end)
    else:
        print("No path found")

ModuleNotFoundError: No module named 'matplotlib'